In [9]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# craeting pdf file with mupltiple tables in multiple pages  
from fpdf import FPDF

# Create a class for the PDF
class PDF(FPDF):
    # def header(self):
    #     self.set_font('Arial', 'B', 12)
    #     self.cell(0, 10, 'PDF with Multiple Tables', 0, 1, 'C')

    # def footer(self):
    #     self.set_y(-15)
    #     self.set_font('Arial', 'I', 8)
    #     self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def create_table(self, title, data):
        # Add a new page
        self.add_page()
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'C')
        self.ln(5)  # Line break

        # Set font for table content
        self.set_font('Arial', '', 10)

        # Table header
        self.cell(40, 10, 'Column 1', 1)
        self.cell(40, 10, 'Column 2', 1)
        self.cell(40, 10, 'Column 3', 1)
        self.ln()

        # Table rows
        for row in data:
            for item in row:
                self.cell(40, 10, str(item), 1)  # Convert to string for safety
            self.ln()

# Create instance of PDF class
pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Data for three tables
tables_data = [
    (f'Table 1', [[f'Row {i+1} Col {j+1}' for j in range(3)] for i in range(100)]),
    (f'Table 2', [[f'Row {i+101} Col {j+1}' for j in range(3)] for i in range(100)]),
    (f'Table 3', [[f'Row {i+201} Col {j+1}' for j in range(3)] for i in range(100)])
]

# Create tables in the PDF
for title, data in tables_data:
    pdf.create_table(title, data)

# Save the PDF to a file
pdf.output('multiple_tables.pdf')

print("PDF created successfully with multiple tables.")


In [49]:
import camelot

Total tables extracted: 1


,0,1,2,3
0,ID,Name,Age,Country
1,1,Alice,30,USA
2,2,Bob,25,Canada
3,3,Charlie,35,UK
4,4,David,28,Australia
5,5,Eve,22,Germany


In [65]:
import camelot
import sqlite3

# Step 1: Read tables from the PDF file
tables = camelot.read_pdf('your_file.pdf', pages='all')  # Use 'all' to extract from all pages

# Assuming you want to work with the first table
df = tables[0].df  # Convert the first table to a DataFrame
print("Extracted DataFrame:")
print(df)

# Step 2: Set proper column names using the first row and check for duplicates
df.columns = df.iloc[0]  # Set the first row as header
df = df[1:]  # Remove the first row from data
df.reset_index(drop=True, inplace=True)  # Reset index

# Rename 'ID' column to 'UserID' to avoid conflict with SQLite primary key if necessary
if 'ID' in df.columns:
    df.rename(columns={'ID': 'UserID'}, inplace=True)

# Print column names for verification
print("Column names:", df.columns.tolist())  # Debugging: Print column names

# Step 3: Connect to SQLite database (this will create the file if it doesn't exist)
conn = sqlite3.connect('extracted_data.db')
cursor = conn.cursor()

# Step 4: Drop existing table if it exists to avoid duplicate column errors
cursor.execute("DROP TABLE IF EXISTS pdf_table")

# Step 5: Generate CREATE TABLE statement dynamically based on DataFrame columns
columns = df.columns.tolist()  # Get column names from DataFrame

# Define all columns as TEXT and ensure no trailing comma
column_definitions = ', '.join([f"{col} TEXT" for col in columns])  

# Create the SQL statement without trailing commas
create_table_sql = f'''
CREATE TABLE IF NOT EXISTS pdf_table (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    {column_definitions}
)
'''

# Debugging: Print the CREATE TABLE SQL statement
print("CREATE TABLE SQL:", create_table_sql)

# Execute the CREATE TABLE statement
try:
    cursor.execute(create_table_sql)
    print("Table created with columns:", columns)
except sqlite3.OperationalError as e:
    print("Error creating table:", e)

# Step 6: Insert data from the DataFrame into the SQLite database
for index, row in df.iterrows():
    # Ensure all row values are converted to strings for insertion
    insert_sql = f'''
    INSERT INTO pdf_table ({', '.join(columns)}) VALUES ({', '.join(['?' for _ in columns])})
    '''
    
    # Convert row data to strings before inserting
    row_data = tuple(map(str, row))  # Convert each value in the row to string

    try:
        cursor.execute(insert_sql, row_data)  # Use tuple(row) to insert row data as strings
    except sqlite3.OperationalError as e:
        print("Error inserting data:", e)

# Commit changes and close the connection
conn.commit()
#conn.close()

print("Data inserted into the database successfully.")

# Optional: Read back from database to verify everything is correct
conn = sqlite3.connect('extracted_data.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM pdf_table")
rows = cursor.fetchall()

print("Data retrieved from database:")
for row in rows:
    print(row)

#conn.close()


Extracted DataFrame:
    0        1    2          3
0  ID     Name  Age    Country
1   1    Alice   30        USA
2   2      Bob   25     Canada
3   3  Charlie   35         UK
4   4    David   28  Australia
5   5      Eve   22    Germany
Column names: ['UserID', 'Name', 'Age', 'Country']
CREATE TABLE SQL: 
CREATE TABLE IF NOT EXISTS pdf_table (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID TEXT, Name TEXT, Age TEXT, Country TEXT
)

Table created with columns: ['UserID', 'Name', 'Age', 'Country']
Data inserted into the database successfully.
Data retrieved from database:
(1, '1', 'Alice', '30', 'USA')
(2, '2', 'Bob', '25', 'Canada')
(3, '3', 'Charlie', '35', 'UK')
(4, '4', 'David', '28', 'Australia')
(5, '5', 'Eve', '22', 'Germany')


In [124]:
# Connect to the SQLite database
conn = sqlite3.connect('extracted_data_all.db')
cursor = conn.cursor()

# Execute a query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the names of all tables
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
sqlite_sequence
pdf_table_1
pdf_table_2
pdf_table_3
pdf_table_4
pdf_table_5
pdf_table_6
pdf_table_7
pdf_table_8
pdf_table_9
pdf_table_10
pdf_table_11
pdf_table_12


In [93]:
# Print column names
conn = sqlite3.connect('extracted_data_all.db')
cursor = conn.cursor()
# cursor.execute("SELECT * FROM pdf_table_1")
# column_names = [description[0] for description in cursor.description]
# print(tuple(column_names))
# rows = cursor.fetchall()
# for row in rows:
#     print(row)

# current bug, not able to identify till when table is extended in multiple pages, currently for every new page new table is craeted in db

In [121]:
#latest version - 
import camelot
import sqlite3
import pandas as pd  # Ensure pandas is imported

# Step 1: Read tables from the PDF file (all pages)
tables = camelot.read_pdf('multiple_tables.pdf', pages='all')  # Use 'all' to extract from all pages

# Connect to SQLite database (this will create the file if it doesn't exist)
conn = sqlite3.connect('extracted_data_all.db')
cursor = conn.cursor()

# Initialize variables
combined_df = None  # To hold combined rows for a single table
table_count = 0  # To count tables

def save_table_to_db(cursor, df, table_count):
    """Function to save DataFrame to SQLite database."""
    
    # Set proper column names using the first row and check for duplicates
    df.columns = df.iloc[0]  # Set the first row as header
    df = df[1:]  # Remove the first row from data
    df.reset_index(drop=True, inplace=True)  # Reset index

    # Rename 'ID' column to 'UserID' to avoid conflict with SQLite primary key if necessary
    if 'ID' in df.columns:
        df.rename(columns={'ID': 'UserID'}, inplace=True)

    print("Column names:", df.columns.tolist())  # Debugging: Print column names

    # Drop existing table if it exists to avoid duplicate column errors
    table_name = f"pdf_table_{table_count + 1}"  # Create a unique table name for each extracted table
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    # Generate CREATE TABLE statement dynamically based on DataFrame columns
    columns = df.columns.tolist()  # Get column names from DataFrame

    # Quote column names to handle spaces and special characters
    quoted_columns = [f'"{col}"' for col in columns]  
    column_definitions = ', '.join([f"{col} TEXT" for col in quoted_columns])  

    # Create the SQL statement without trailing commas
    create_table_sql = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        {column_definitions}
    )
    '''

    # Debugging: Print the CREATE TABLE SQL statement
    print("CREATE TABLE SQL:", create_table_sql)

    # Execute the CREATE TABLE statement
    try:
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created with columns:", columns)
    except sqlite3.OperationalError as e:
        print("Error creating table:", e)

    # Insert data from the DataFrame into the SQLite database
    for index, row in df.iterrows():
        insert_sql = f'''
        INSERT INTO {table_name} ({', '.join(quoted_columns)}) VALUES ({', '.join(['?' for _ in columns])})
        '''
        
        # Convert row data to strings before inserting
        row_data = tuple(map(str, row))  # Convert each value in the row to string

        try:
            cursor.execute(insert_sql, row_data)  # Use tuple(row) to insert row data as strings
        except sqlite3.OperationalError as e:
            print("Error inserting data:", e)

# Step 2: Iterate over each extracted table
for i, table in enumerate(tables):
    df = table.df  # Convert the current table to a DataFrame
    print(f"Extracted DataFrame for Table {i + 1}:")
    #print(df)

    # Check if the first row contains likely headings (indicating new table)
    first_row = df.iloc[0].values
    
    # Check if any of the values in the first row are strings (indicating potential headings)
    is_heading_row = any(isinstance(item, str) and item.strip() != "" for item in first_row)

    if is_heading_row:
        print("Detected new table based on content in the first row.")
        
        # If we have accumulated rows in combined_df, save it before starting a new one
        if combined_df is not None:
            save_table_to_db(cursor, combined_df, table_count)
            combined_df = None  
            table_count += 1  # Increment table count

        combined_df = df.copy()  # Start with this DataFrame as it contains headers

    else:
        # If it's not a heading row and we already have a combined_df, append it
        if combined_df is not None:
            combined_df = pd.concat([combined_df, df], ignore_index=True)

# After looping through all tables, save any remaining data
if combined_df is not None:
    save_table_to_db(cursor, combined_df, table_count)

# Commit changes and close the connection
conn.commit()
conn.close()

print("All tables inserted into the database successfully.")


Extracted DataFrame for Table 1:
               0             1             2
0       Column 1      Column 2      Column 3
1    Row 1 Col 1   Row 1 Col 2   Row 1 Col 3
2    Row 2 Col 1   Row 2 Col 2   Row 2 Col 3
3    Row 3 Col 1   Row 3 Col 2   Row 3 Col 3
4    Row 4 Col 1   Row 4 Col 2   Row 4 Col 3
5    Row 5 Col 1   Row 5 Col 2   Row 5 Col 3
6    Row 6 Col 1   Row 6 Col 2   Row 6 Col 3
7    Row 7 Col 1   Row 7 Col 2   Row 7 Col 3
8    Row 8 Col 1   Row 8 Col 2   Row 8 Col 3
9    Row 9 Col 1   Row 9 Col 2   Row 9 Col 3
10  Row 10 Col 1  Row 10 Col 2  Row 10 Col 3
11  Row 11 Col 1  Row 11 Col 2  Row 11 Col 3
12  Row 12 Col 1  Row 12 Col 2  Row 12 Col 3
13  Row 13 Col 1  Row 13 Col 2  Row 13 Col 3
14  Row 14 Col 1  Row 14 Col 2  Row 14 Col 3
15  Row 15 Col 1  Row 15 Col 2  Row 15 Col 3
16  Row 16 Col 1  Row 16 Col 2  Row 16 Col 3
17  Row 17 Col 1  Row 17 Col 2  Row 17 Col 3
18  Row 18 Col 1  Row 18 Col 2  Row 18 Col 3
19  Row 19 Col 1  Row 19 Col 2  Row 19 Col 3
20  Row 20 Col 1  Row 